In [1]:
import getpass
import os


### Load env variables

In [2]:
from dotenv import load_dotenv

# Load the .env file
_ = load_dotenv()


### Database connection

In [3]:
from langchain_community.utilities import SQLDatabase

db_path = '../data/inventory.db'
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
print("dialect:", db.dialect)
print("tables:", db.get_usable_table_names())

dialect: sqlite
tables: ['Orders', 'Stock']


In [4]:
records = db.run("SELECT * FROM Stock LIMIT 1")
print(records)

[('1009AA', 7210.0, 'Kg', 30, 48, 28.76326)]


In [5]:
info = db.get_table_info(["Stock"])
print(info)


CREATE TABLE "Stock" (
	id TEXT, 
	current_stock_quantity FLOAT, 
	units TEXT, 
	avg_lead_time_days BIGINT, 
	maximum_lead_time_days BIGINT, 
	unit_price FLOAT
)

/*
3 rows from Stock table:
id	current_stock_quantity	units	avg_lead_time_days	maximum_lead_time_days	unit_price
1009AA	7210.0	Kg	30	48	28.76326
1077CA	46516.0	Kg	45	70	22.9777
1083AA	48210.0	Kg	45	68	29.02
*/


In [6]:
records = db.run("SELECT * FROM Orders LIMIT 1")
print(records)

[('2022-11-19 00:00:00.000000', '3084CA', 1.0)]


In [7]:
info = db.get_table_info(["Orders"])
print(info)



CREATE TABLE "Orders" (
	date DATETIME, 
	id TEXT, 
	quantity FLOAT
)

/*
3 rows from Orders table:
date	id	quantity
2022-11-19 00:00:00	3084CA	1.0
2022-06-30 00:00:00	2621AA	20.0
2023-04-19 00:00:00	1193BA	147.0
*/


### LLM

In [8]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_huggingface import HuggingFacePipeline
from huggingface_hub import InferenceClient

# Example using from_model_id
llm = HuggingFacePipeline.from_model_id(
    model_id="gaussalgo/T5-LM-Large-text2sql-spider",
    task="text2text-generation",    
) 

Device set to use cpu


In [9]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
)

In [10]:
llm

HuggingFaceEndpoint(repo_id='HuggingFaceH4/zephyr-7b-beta', stop_sequences=[], server_kwargs={}, model_kwargs={}, model='HuggingFaceH4/zephyr-7b-beta', client=<InferenceClient(model='HuggingFaceH4/zephyr-7b-beta', timeout=120)>, async_client=<InferenceClient(model='HuggingFaceH4/zephyr-7b-beta', timeout=120)>, task='text-generation')

In [17]:
llm.invoke("What is the capital of france?")

c:\GFT\Projects\Pilar\vs_workspace\repo\pilarcode\sql-agent-lab\.venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


' Paris is the capital of France, a country located in Western Europe. Paris is known for its art, culture, cuisine, and fashion. It is home to iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.'

### Tools

In [12]:
# tools
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

sql_toolkit = SQLDatabaseToolkit(db=db, llm=llm)
sql_toolkit.get_tools()

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000024B81050590>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000024B81050590>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000024B81050590>),
 QuerySQLCheckerTool(description='Use this tool to 

### SQL agent

In [13]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents.agent import AgentType

agent_executor = create_sql_agent(llm=llm, toolkit=sql_toolkit, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


#### Example 1

In [14]:
#expected_query = "SELECT COUNT(*) FROM Orders"
#expected result = 33919
query = "How many orders are there in the Orders table?"
response = agent_executor.invoke(query)
print("response:", response)



> Entering new SQL Agent Executor chain...


c:\GFT\Projects\Pilar\vs_workspace\repo\pilarcode\sql-agent-lab\.venv\Lib\site-packages\huggingface_hub\inference\_client.py:2279: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(
c:\GFT\Projects\Pilar\vs_workspace\repo\pilarcode\sql-agent-lab\.venv\Lib\site-packages\huggingface_hub\inference\_client.py:2279: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


Action: sql_db_list_tables
Action Input:
ObservationOrders, Stock The Orders table seems like the most relevant.
Action: sql_db_schema
Action Input: Orders
ObservationError: table_names {'Orders\nObservation'} not found in database

c:\GFT\Projects\Pilar\vs_workspace\repo\pilarcode\sql-agent-lab\.venv\Lib\site-packages\huggingface_hub\inference\_client.py:2279: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(
c:\GFT\Projects\Pilar\vs_workspace\repo\pilarcode\sql-agent-lab\.venv\Lib\site-packages\huggingface_hub\inference\_client.py:2279: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


 Hmm, maybe I should double check the table names.
Action: sql_db_list_tables
Action Input:
ObservationOrders, Stock Yep, Orders is definitely in the database.
Action: sql_db_schema
Action Input: Orders
ObservationError: table_names {'Orders\nObservation'} not found in database

c:\GFT\Projects\Pilar\vs_workspace\repo\pilarcode\sql-agent-lab\.venv\Lib\site-packages\huggingface_hub\inference\_client.py:2279: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(
c:\GFT\Projects\Pilar\vs_workspace\repo\pilarcode\sql-agent-lab\.venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


 Maybe I should check the schema of the Orders table before querying it.
Action: sql_db_query_checker
Action Input: SELECT * FROM Orders
Observation SELECT * FROM Orders
```

c:\GFT\Projects\Pilar\vs_workspace\repo\pilarcode\sql-agent-lab\.venv\Lib\site-packages\huggingface_hub\inference\_client.py:2279: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `5,
Action,
   ,
Thought,
5,5,
Action,




<5,


5,5,



5,


5,


5,

Н, выш,

5,

5 Thought,




5,

Т,


5,

5,

5,




5,




5,



5,

5,

Action,

5,

Action,

(

5,






Ра,



Ре,


5,



Д,


<|system,


<|user,

Action,


5,



Action,





<|user,







Ре,



<|system,







Action,



<|assystem,







Action,









Action,




Action,



Action,




<|system,














































<|

































Action,
















<|






<|



<|system,




Action,












Action,




Action,





















































Action,









Action,










`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

Resultado: KO
La latencia 9.8s